In [ ]:
import numpy as np
from keras.layers import Input, Convolution2D, Dense, Dropout, Flatten, concatenate, BatchNormalization
from keras.models import Model  # basic class for specifying and training a neural network
from keras import losses
import keras

import tensorflow as tf

In [ ]:
SIZE = 9
CHANNELS = 4

In [ ]:
npz = np.load('gomoku/models/9_vectors.npz')
train_p_vectors = npz['train_p_vectors']
train_q_vectors = npz['train_q_vectors']
train_p = npz['train_p']
train_q = npz['train_q']

In [ ]:
inp = Input(shape=(SIZE, SIZE, CHANNELS))

# key difference between this and conv network is padding
conv_1 = Convolution2D(64, (3, 3), padding='same', activation='relu',
                       kernel_initializer='random_normal', use_bias=False)(inp)
bn2 = BatchNormalization()(conv_1)
conv_2 = Convolution2D(32, (3, 3), padding='valid', activation='relu',
                       kernel_initializer='random_normal', use_bias=False)(bn2)
bn3 = BatchNormalization()(conv_2)
conv_3 = Convolution2D(32, (3, 3), padding='valid', activation='relu',
                       kernel_initializer='random_normal', use_bias=False)(bn3)
bn4 = BatchNormalization()(conv_3)
conv_4 = Convolution2D(16, (3, 3), padding='valid', activation='relu',
                       kernel_initializer='random_normal', use_bias=False)(bn4)
bn5 = BatchNormalization()(conv_4)
conv_5 = Convolution2D(8, (3, 3), padding='valid', activation='relu',
                       kernel_initializer='random_normal', use_bias=False)(bn5)
bn6 = BatchNormalization()(conv_5)

flat = Flatten()(bn5)

hidden = Dense(10, activation='relu', kernel_initializer='random_normal', use_bias=False)(flat)
bn_final = BatchNormalization()(hidden)

out = Dense(1, use_bias=False)(bn_final)

q_model = Model(inputs=[inp], outputs=out)
q_model.compile(loss=losses.mean_squared_error, optimizer='adam', metrics=['mean_squared_error'])


In [ ]:
q_model.fit(train_q_vectors,
            train_q, 
            shuffle=True,
            epochs=1000,
            batch_size=100,
            validation_split=0.1)

In [ ]:
q_model.save('gomoku/models/waiting_q.model')

In [ ]:
inp = Input(shape=(SIZE, SIZE, CHANNELS))

conv_1 = Convolution2D(64, (3, 3), padding='same', activation='relu',
                       kernel_initializer='random_normal', use_bias=False)(inp)
bn2 = BatchNormalization()(conv_1)
conv_2 = Convolution2D(32, (3, 3), padding='same', activation='relu',
                       kernel_initializer='random_normal', use_bias=False)(bn2)
bn3 = BatchNormalization()(conv_2)
conv_3 = Convolution2D(16, (3, 3), padding='same', activation='relu',
                       kernel_initializer='random_normal', use_bias=False)(bn3)
bn4 = BatchNormalization()(conv_3)
conv_4 = Convolution2D(8, (3, 3), padding='same', activation='relu',
                       kernel_initializer='random_normal', use_bias=False)(bn4)
bn5 = BatchNormalization()(conv_4)
#conv_5 = Convolution2D(64, (3, 3), padding='same', activation='relu',
#                       kernel_initializer='random_uniform')(bn5)
#bn6 = BatchNormalization()(conv_5)

flat = Flatten()(bn5)

hidden = Dense(SIZE ** 2, activation='relu', kernel_initializer='random_normal', use_bias=False)(flat)
bn_final = BatchNormalization()(hidden)

out = Dense(SIZE ** 2, activation='softmax')(bn_final)

p_model = Model(inputs=[inp], outputs=out)
p_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
with tf.device('/gpu:0'):
    p_model.fit(train_p_vectors, 
                train_p,
                shuffle=True,
                batch_size=100,
              epochs=100,
             validation_split=0.1)

In [ ]:
p_model.save('models/9_9_p_v2_2.net')